# Sae Cryptographie 

## Partie 1

Extention du Code SDES 

In [1]:
from sys import exit
from time import time
 
KeyLength = 10
SubKeyLength = 8
DataLength = 8
FLength = 4
 
# Tables for initial and final permutations (b1, b2, b3, ... b8)
IPtable = (2, 6, 3, 1, 4, 8, 5, 7)
FPtable = (4, 1, 3, 5, 7, 2, 8, 6)
 
# Tables for subkey generation (k1, k2, k3, ... k10)
P10table = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8table = (6, 3, 7, 4, 8, 5, 10, 9)
 
# Tables for the fk function
EPtable = (4, 1, 2, 3, 2, 3, 4, 1)
S0table = (1, 0, 3, 2, 3, 2, 1, 0, 0, 2, 1, 3, 3, 1, 3, 2)
S1table = (0, 1, 2, 3, 2, 0, 1, 3, 3, 0, 1, 0, 2, 1, 0, 3)
P4table = (2, 4, 3, 1)
 
def perm(inputByte, permTable):
    """Permute input byte according to permutation table"""
    outputByte = 0
    for index, elem in enumerate(permTable):
        if index >= elem:
            outputByte |= (inputByte & (128 >> (elem - 1))) >> (index - (elem - 1))
        else:
            outputByte |= (inputByte & (128 >> (elem - 1))) << ((elem - 1) - index)
    return outputByte
 
def ip(inputByte):
    """Perform the initial permutation on data"""
    return perm(inputByte, IPtable)
 
def fp(inputByte):
    """Perform the final permutation on data"""
    return perm(inputByte, FPtable)
 
def swapNibbles(inputByte):
    """Swap the two nibbles of data"""
    return (inputByte << 4 | inputByte >> 4) & 0xff
 
def keyGen(key):
    """Generate the two required subkeys"""
    def leftShift(keyBitList):
        """Perform a circular left shift on the first and second five bits"""
        shiftedKey = [None] * KeyLength
        shiftedKey[0:9] = keyBitList[1:10]
        shiftedKey[4] = keyBitList[0]
        shiftedKey[9] = keyBitList[5]
        return shiftedKey
 
    # Converts input key (integer) into a list of binary digits
    keyList = [(key & 1 << i) >> i for i in reversed(range(KeyLength))]
    permKeyList = [None] * KeyLength
    for index, elem in enumerate(P10table):
        permKeyList[index] = keyList[elem - 1]
    shiftedOnceKey = leftShift(permKeyList)
    shiftedTwiceKey = leftShift(leftShift(shiftedOnceKey))
    subKey1 = subKey2 = 0
    for index, elem in enumerate(P8table):
        subKey1 += (128 >> index) * shiftedOnceKey[elem - 1]
        subKey2 += (128 >> index) * shiftedTwiceKey[elem - 1]
    return (subKey1, subKey2)
 
def fk(subKey, inputData):
    """Apply Feistel function on data with given subkey"""
    def F(sKey, rightNibble):
        aux = sKey ^ perm(swapNibbles(rightNibble), EPtable)
        index1 = ((aux & 0x80) >> 4) + ((aux & 0x40) >> 5) + \
                 ((aux & 0x20) >> 5) + ((aux & 0x10) >> 2)
        index2 = ((aux & 0x08) >> 0) + ((aux & 0x04) >> 1) + \
                 ((aux & 0x02) >> 1) + ((aux & 0x01) << 2)
        sboxOutputs = swapNibbles((S0table[index1] << 2) + S1table[index2])
        return perm(sboxOutputs, P4table)
 
    leftNibble, rightNibble = inputData & 0xf0, inputData & 0x0f
    return (leftNibble ^ F(subKey, rightNibble)) | rightNibble
 
def encrypt(key, plaintext):
    """Encrypt plaintext with given key"""
    data = fk(keyGen(key)[0], ip(plaintext))
    return fp(fk(keyGen(key)[1], swapNibbles(data)))
 
def decrypt(key, ciphertext):
    """Decrypt ciphertext with given key"""
    data = fk(keyGen(key)[1], ip(ciphertext))
    return fp(fk(keyGen(key)[0], swapNibbles(data)))  
 
if __name__ == '__main__':
    # Test vectors described in "Simplified DES (SDES)"
    # (http://www2.kinneret.ac.il/mjmay/ise328/328-Assignment1-SDES.pdf)
 
    try:
        assert encrypt(0b0000000000, 0b10101010) == 0b00010001
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b0000000000, 0b10101010), "Expected: ", 0b00010001)
        exit(1)
    try:
        assert encrypt(0b1110001110, 0b10101010) == 0b11001010
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1110001110, 0b10101010), "Expected: ", 0b11001010)
        exit(1)
    try:
        assert encrypt(0b1110001110, 0b01010101) == 0b01110000
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1110001110, 0b01010101), "Expected: ", 0b01110000)
        exit(1)
    try:
        assert encrypt(0b1111111111, 0b10101010) == 0b00000100
    except AssertionError:
        print("Error on encrypt:")
        print("Output: ", encrypt(0b1111111111, 0b10101010), "Expected: ", 0b00000100)
        exit(1)
 
    t1 = time()
    for i in range(1000):
        encrypt(0b1110001110, 0b10101010)
    t2 = time()
    print("Elapsed time for 1,000 encryptions: {:0.3f}s".format(t2 - t1))

Elapsed time for 1,000 encryptions: 0.030s


Extention du protocole double SDES

In [2]:
# Chiffrer comme les SDES (chiffré 2 fois avec deux clés différentes) une liste de 8 bits
def chiffrer(k1,k2,msg):
    cipher = []
    for b in msg:
        cipher.append(encrypt(k1,encrypt(k2,b)))
    return cipher

# Déchiffrer comme les SDES (déchiffré 2 fois avec deux clés différentes)
def dechiffrer(k1,k2,cipher):
    msg = []
    for b in cipher:
        msg.append(decrypt(k2,decrypt(k1,b)))
    return msg

# Transforme un message en une liste de 8 bits
def msg_to_list(msg):
    return [ord(c) for c in msg]

# Transforme une liste de 8 bits en un message
def list_to_msg(msg):
    return "".join([chr(c) for c in msg])

with open("arsene_lupin_extrait.txt", "r") as f:
    msg = f.read()
    cipher = chiffrer(0b110100001,0b011011010, msg_to_list(msg))
    print(cipher)
    msg = list_to_msg(dechiffrer(0b110100001,0b011011010, cipher))
    print(msg)


[253, 238, 8, 83, 199, 142, 83, 210, 83, 199, 142, 64, 187, 247, 142, 210, 251, 41, 210, 50, 247, 236, 64, 83, 210, 252, 155, 64, 111, 55, 224, 83, 210, 52, 41, 15, 247, 224, 114, 210, 185, 83, 224, 142, 50, 83, 243, 187, 224, 179, 189, 187, 243, 188, 64, 247, 181, 50, 83, 41, 64, 252, 210, 251, 83, 210, 3, 187, 41, 64, 247, 189, 83, 210, 52, 83, 188, 50, 187, 224, 189, 85, 253, 210, 9, 181, 41, 64, 189, 83, 116, 210, 50, 83, 210, 15, 64, 181, 86, 83, 142, 210, 59, 41, 142, 83, 224, 188, 83, 64, 185, 210, 183, 142, 142, 15, 111, 116, 176, 176, 157, 157, 157, 18, 185, 41, 142, 83, 224, 188, 83, 64, 185, 18, 181, 64, 185, 176, 83, 188, 181, 181, 29, 111, 176, 152, 168, 226, 254, 206, 85, 85, 155, 64, 111, 55, 224, 83, 210, 52, 41, 15, 247, 224, 210, 15, 187, 64, 243, 247, 210, 224, 181, 41, 111, 126, 210, 50, 184, 247, 224, 111, 187, 247, 111, 247, 111, 111, 187, 188, 50, 83, 210, 189, 187, 243, 188, 64, 247, 181, 50, 83, 41, 64, 210, 251, 181, 224, 142, 210, 181, 224, 210, 64, 187, 189,

Code du cassage_brutal

In [37]:
def cassage_brutal(message_clair, message_chiffre):
    nbit = 0
    for k1 in range(0, 1024):
        for k2 in range(0, 1024):
            nbit += 1
            if chiffrer(k1, k2, message_clair) == message_chiffre:
                return (k1, k2), nbit
    return None

# print(cassage_brutal(msg_to_list("Bonjour"), chiffrer(0b110100001,0b011011010,msg_to_list("Bonjour"))))
# print(0b110100001,0b011011010)

# with open("arsene_lupin_extrait.txt", "r") as f:
#     msg = f.read()
msg = "Bonjour Chef"
cipher = chiffrer(500, 1000, msg_to_list(msg))
print(cassage_brutal(msg_to_list(msg), cipher))
print(22, 17)

((500, 1000), 513001)
22 17


Code du cassage_astucieux

In [36]:
def cassage_astucieux(message_clair, message_chiffre):
    nbit = 0
            
    for k1 in range(0, 1024):
        for k2 in range(k1, 1024):
            nbit += 1
            if dechiffrer(k2, k1, message_chiffre[:11]) == message_clair[:11]:
                return (k2, k1), nbit
            if dechiffrer(k1, k2, message_chiffre[:11]) == message_clair[:11]:
                return (k1, k2), nbit

    return None

# decrypted_message = dechiffrer(k1, k2, message_chiffre)
            
# # Ne prendre en compte que les clés qui permettent d'obtenir les trois premiers caractères
# if decrypted_message[:3] == message_clair[:3]:

msg = "Bonjour Chef"
cipher = chiffrer(500, 1000, msg_to_list(msg))
print(cassage_astucieux(msg_to_list(msg), cipher))
print(22, 17)

((500, 1000), 387751)
22 17


Test sur tout le message Arsène Lupin


In [ ]:
msg = """
# Texte extrait du livre "Arsène Lupin, gentleman-cambrioleur" de Maurice Leblanc
# Source: le projet Gutenberg https://www.gutenberg.org/ebooks/32854

Arsène Lupin parmi nous! l'insaisissable cambrioleur dont on racontait
les prouesses dans tous les journaux depuis des mois! l'énigmatique
personnage avec qui le vieux Ganimard, notre meilleur policier, avait
engagé ce duel à mort dont les péripéties se déroulaient de façon si
pittoresque! Arsène Lupin, le fantaisiste gentleman qui n'opère que
dans les châteaux et les salons, et qui, une nuit, où il avait pénétré
chez le baron Schormann, en était parti les mains vides et avait
laissé sa carte, ornée de cette formule: «Arsène Lupin,
gentleman-cambrioleur, reviendra quand les meubles seront
authentiques». Arsène Lupin, l'homme aux mille déguisements: tour à
tour chauffeur, ténor, bookmaker, fils de famille, adolescent,
vieillard, commis-voyageur marseillais, médecin russe, torero
espagnol!
"""

cipher = chiffrer(22, 17, msg_to_list(msg))
print(cassage_astucieux(msg_to_list(msg), cipher))

## Partie 2

In [14]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import padding
import os

# Générer une clé AES-256 bits de manière sécurisée
def generate_aes_key():
    password = b'your_secret_password'  # Remplacez par un mot de passe sécurisé
    salt = os.urandom(16)  # Génère une valeur aléatoire pour le sel
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 256 bits
        salt=salt,
        iterations=100000,  # Choisissez un nombre approprié d'itérations
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key


def encrypt_aes(msg, key):
    encryptor = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).encryptor()
    if isinstance(msg, str):
        msg = msg.encode('utf-8')  # Convertir la chaîne en octets

    # Ajouter du bourrage (padding)
    padder = padding.PKCS7(128).padder()
    padded_msg = padder.update(msg) + padder.finalize()

    return encryptor.update(padded_msg) + encryptor.finalize()

# Déchiffrer un message avec AES-256 bits
def decrypt_aes(msg, key):
    decryptor = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend()).decryptor()
    decrypted_msg = decryptor.update(msg) + decryptor.finalize()

    # Retirer le bourrage
    unpadder = padding.PKCS7(128).unpadder()
    unpadded_msg = unpadder.update(decrypted_msg) + unpadder.finalize()

    return unpadded_msg.decode('utf-8')

# Utiliser les fonctions
key_aes = generate_aes_key()
cipher_text = encrypt_aes("Bonjour chef", key_aes)
print(cipher_text)

plain_text = decrypt_aes(cipher_text, key_aes)
print(plain_text)


b'\x8e\x94\xea\xb2\xb0W\xe2\xc5\xb8\xb6\x1f\xa3#r\xea\xe7'
Bonjour chef


In [26]:
import time

message = "Bonjour chef"
cle = generate_aes_key()

# Mesurer le temps d'exécution du chiffrement AES
start_time = time.time()
cipher_aes = encrypt_aes(message, cle)
encryption_time_aes = time.time() - start_time
print("Temps d'exécution du chiffrage AES:", encryption_time_aes*1000)

# Mesurer le temps d'exécution du déchiffrement AES
start_time = time.time()
decipher_aes = decrypt_aes(cipher_aes, cle)
decryption_time_aes = time.time() - start_time
print("Temps d'exécution du déchiffrage AES:", decryption_time_aes*1000)

# Mesurer le temps d'exécution du chiffrement SDES
start_time = time.time()
cipher_sdes = chiffrer(0b110100001, 0b011011010, msg_to_list(message))
encryption_time_sdes = time.time() - start_time
print("Temps d'exécution du chiffrage SDES:", encryption_time_sdes*1000)

# Mesurer le temps d'exécution du déchiffrement SDES
start_time = time.time()
decipher_sdes = dechiffrer(0b110100001, 0b011011010, cipher_sdes)
decryption_time_sdes = time.time() - start_time
print("Temps d'exécution du déchiffrage SDES:", decryption_time_sdes*1000)


Temps d'exécution du chiffrage AES: 0.17070770263671875
Temps d'exécution du déchiffrage AES: 0.3566741943359375
Temps d'exécution du chiffrage SDES: 0.6959438323974609
Temps d'exécution du déchiffrage SDES: 0.9150505065917969


In [1]:
Partie image

dans la partie changer il y a un partie de différent dans cette partie il y a des pixel qui change et qui on comme valeur entre 0-256
en la transformant un bits 0000 0000 seule le dernier est rempléce 0000 0001 
en additionnant bout a bout on obtiens la clé.

! il faut analyser les dernier bits de chaque pixel.

clé = suite de bits insérer dans l'image

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1196515409.py, line 4)

In [9]:
from PIL import Image

def average_pixel_value(image_path):
    image = Image.open(image_path)
    pixels = list(image.getdata())
    average_value = sum(pixels) / len(pixels)
    return average_value

print(average_pixel_value("rossignol2.bmp"))
print(average_pixel_value("rossignol1.bmp"))

169.43496337890625
169.9302001953125


In [26]:
from PIL import Image

def comparer_images(image2_path):
    # Ouvrir les deux images
    img2 = Image.open(image2_path)

    # Créer une chaîne binaire pour représenter les différences de pixels
    differences = ""

    # Parcourir tous les pixels des deux images
    for x in range(64):
        # Récupérer les valeurs de pixel correspondantes dans chaque image
        pixel2 = img2.getpixel((0, x))

        # Comparer les pixels et ajouter 1 ou 0 à la chaîne binaire
        differences += str(pixel2%2)


    # Retourner la chaîne binaire représentant les différences
    return differences

# Exemple d'utilisation
image1_path = "rossignol1.bmp"

# Exemple d'utilisation de la fonction avec deux images spécifiques
resultat = comparer_images(image1_path)


print("Clé extraite :", resultat)

Clé extraite : 0001111100000111100010000100001011001011010010001100100110001001


## Partie3